## A python version of the night-time lights image.

This notebook is a quick example of some band math and custom visualization.  
Based on example found [here](https://github.com/google/earthengine-api)  
13 Apr 2022  
NSh  


In [ ]:
# Import, initialize, and authenticate the GEE API.
# You will be redirected to another tab. Copy the authentication code and paste below when prompted.
import ee
import collections
collections.Callable = collections.abc.Callable
ee.Authenticate()

Uncomment the following, run the previous and following cells, and then follow the directions (restart kernel and re-comment the following cell.) You should only have to do this once.

In [ ]:
geemap.update_package()

In [ ]:
# Initialize and import packages.
import geemap
ee.Initialize()

The following cell defines a function. It will not produce any output.

In [ ]:
#Compute the trend of night-time lights.

# Add a band containing image date as years since 1991.
def create_time_band(img):
    year = ee.Date(img.get('system:time_start')).get('year').subtract(1991)
    return ee.Image(year).byte().addBands(img)


The following cell imports a product by NOAA and stores it as a collection of images. Then, it "maps" (applies) the function we defined above to all the images in the collection. This cell does not produce visible output.

In [ ]:
# Map the time band creation helper over the night-time lights collection.
# https://developers.google.com/earth-engine/datasets/catalog/NOAA_DMSP-OLS_NIGHTTIME_LIGHTS
collection = (ee.ImageCollection('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS')
    .select('stable_lights')
    .map(create_time_band))


The following cell will produce an interactive map.

In [ ]:
# Compute a linear fit over the series of values at each pixel, 
# visualizing the y-intercept in green, and positive/negative slopes as red/blue.
# Create a map centered at (lat, lon).
Map = geemap.Map(center=[50, 5], zoom=4)
Map.addLayer(
    collection.reduce(ee.Reducer.linearFit()),
    {'min': 0, 'max': [0.18, 20, -0.18], 'bands': ['scale', 'offset', 'scale']},
    'stable lights trend')

# Display map
Map

